In [1]:
import pandas as pd
import numpy as np
import Levenshtein as lev
import re
from simhash import Simhash

In [20]:
duplist = pd.read_csv('with_dup_id')

In [3]:
duplist.columns

Index(['id', 'location', 'ask', 'bedrooms', 'title', 'address', 'posting_date',
       'created_at', 'updated_at', 'source_id', 'survey_id', 'payload',
       'dup_t', 'dup_tb', 'dup_tba', 'dup_tbl', 'dup_tbal'],
      dtype='object')

In [21]:
duplist['sim'] = np.zeros(len(duplist.index))

In [22]:
def get_features(s):
    width = 3
    s = s.lower()
    s = re.sub(r'[^\w]+', '', s)
    return [s[i:i + width] for i in range(max(len(s) - width + 1, 1))]

In [25]:
for k in range(0, len(duplist.index)) :
    duplist.loc[k, 'sim'] = bin(Simhash(get_features(duplist.loc[k, 'title'])).value)

In [26]:
duplist = duplist.sort_values(by='sim')
duplist = duplist.reset_index(drop=True)

In [14]:
def condition(row, r):
    return lev.hamming(duplist.loc[row, 'sim'], duplist.loc[r, 'sim']) <= 5

In [3]:
i = 0
row = 0
while row < len(duplist.index) :
    duplist.loc[row, 'dup_tbal'] = i
    j = 1
    r = row + j
    while (r < len(duplist.index)) and condition(row, r) :
        duplist.loc[r, 'dup_tbal'] = i
        j += 1
        r = row + j
    row = r
    i += 1

In [ ]:
duplist.to_csv('with_dup_id_2')